In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pyathena as pa
import pandas as pd
import xarray as xr
import glob
import os

In [102]:
import astropy.constants as c
import astropy.units as u
unit=pa.set_units(muH=1.4271)

plt.rcParams['font.size']=16
import os,glob
labels=['noICM_new','ICM1_new','ICM2_new'] # ['no ICM','ICM00','ICM0']#,'ICM00','ICM0',,'ICM2','ICM3','ICM4'
ids=['RPS_8pc_noICM_newacc','RPS_8pc_ICM0_newacc','RPS_8pc_ICM1_newacc','RPS_8pc_ICM2_newacc','RPS_8pc_ICM3_newacc'] # RPS_8pc_ICM00','RPS_8pc_ICM0',,'RPS_8pc_ICM2','RPS_8pc_ICM3','RPS_8pc_ICM4'
#ids=['R8_8pc_metal','RPS_8pc_ICM0','RPS_8pc_ICM1','RPS_8pc_ICM2','RPS_8pc_ICM3']
base='G:/yeongu/'
#base='F:/yeongu/'
#base2='D:/yeongu/'
#C = ('darkkhaki','b','r','k','g','magenta','cyan') # 
C = ('salmon','r','deepskyblue','b','g')
C2 = ('darkblue','deepskyblue','goldenrod','red','firebrick')
hh = [0.006,0.006,0.15]
labell = ('No ICM','Very Weak', 'Weak', 'Strong','Very Strong')
#fig =plt.figure(figsize=(7.5,12))
for j,problem_id in enumerate(ids):
    #rstfiles=glob.glob('%s%s/id0/%s.????.rst' % (base,problem_id,problem_id))
    #rstfiles+=glob.glob('%s%s/rst/%s.????.rst' % (base,problem_id,problem_id))
    parfile='{}/{}/{}.par'.format(base,problem_id,problem_id)
    hstfile='{}/{}/hst/{}.hst'.format(base,problem_id,problem_id)
    hst=pa.hst_reader(hstfile)

    Myr=unit['time'].to('Myr').value
    print unit['density']
    hst.index=hst.time
    hst=hst.loc[250:500,:]
    hst.index=hst.time*Myr
    par=pa.get_params(parfile)
    
    Lx=par['x1max']-par['x1min']
    Ly=par['x2max']-par['x2min']
    Lz=par['x3max']-par['x3min']
    
    plist=['phase1','phase2','phase3','phase4','phase5']
    datasets = xr.Dataset()
    for phase in plist:
        path='{}{}/zprof_merged/{}.{}.zprof.nc'.format(base,problem_id,problem_id,phase)
        with xr.open_dataarray(path) as da: da.load()
        #print da
        datasets[phase]=da
    zprof_ds=datasets
    zprof_ds = zprof_ds.sel(taxis=slice(250,500))
    #print zprof_ds.fields
    Munit=(unit['density']*unit['velocity']*unit['time']).to('Msun/pc^2').value
    #print zprof_ds['phase1'].loc['A'].data.cumsum(), zprof_ds['phase2'].loc['A']
    zprof = zprof_ds.to_array().sum(dim='variable')
    #print zprof
    #print zprof.loc['ge']
    print unit['gravitational_potential']
    phiex = zprof.loc['Phie'].data*unit['gravitational_potential']/(1024*1024)
    gext = zprof.loc['gext'].data/(1024.*1024)*(3.24*1e-14) # km/s^2
    gsg = zprof.loc['gsg'].data/(1024.*1024)*(3.24*1e-14)
    d = zprof.loc['d'].data*unit['density']/(1024.*1024)*(8*8*8)
    print d.max(), d.min()
    #d = zprof.loc['d'][:,0].data*unit['density']*(8*8*8)
    #print np.sum(d)/(1024*1024*8)
    z = zprof.zaxis/1000.
    #plt.plot(z,phiex)
    #plt.plot(z,gext)
    #plt.plot(z,gsg)
    #plt.plot(z,d)
    for i in range(249):
        plt.plot(z,(d*gext)[:,i],c='r',label='External')
        plt.plot(z,(d*gsg)[:,i],c='g',label='Self')
    
        plt.plot(z,(d*gext)[:,i][::-1].cumsum()[::-1],c='r',ls='--')
        plt.plot(z,(d*gsg)[:,i][::-1].cumsum()[::-1],c='g',ls='--')
        plt.xlim(z.min(),z.max())
        plt.ylim(-1e-12,2e-11)
        plt.xlabel('z [kpc]')
        plt.legend(loc=0)
        plt.savefig('D:/yeongu/plots/zprof/%s_%s.png' % (problem_id,i))
        plt.close()
        #plt.show()
    
    '''
    plt.subplot(5,1,j+1)
    if j==0:
        cold = ((zprof_ds['phase1'].loc['M3'] + zprof_ds['phase2'].loc['M3'] + zprof_ds['phase3'].loc['M3'])[-1,:].data-(zprof_ds['phase1'].loc['M3'] + zprof_ds['phase2'].loc['M3'] + zprof_ds['phase3'].loc['M3'])[0,:].data)*Munit/(1024*1024)/Myr
        ion = ((zprof_ds['phase4'].loc['M3'])[-1,:].data-(zprof_ds['phase4'].loc['M3'])[0,:].data)*Munit/(1024*1024)/Myr
        hot = ((zprof_ds['phase5'].loc['M3'])[-1,:].data-(zprof_ds['phase5'].loc['M3'])[0,:].data)*Munit/(1024*1024)/Myr
    else:
        cold = (zprof_ds['phase1'].loc['M3'] + zprof_ds['phase2'].loc['M3'] + zprof_ds['phase3'].loc['M3'])[-1,:].data*Munit/(1024*1024)/Myr
        ion = (zprof_ds['phase4'].loc['M3'])[-1,:].data*Munit/(1024*1024)/Myr
        hot = (zprof_ds['phase5'].loc['M3'])[-1,:].data*Munit/(1024*1024)/Myr
        #icm = (zprof_ds.to_array().sum(dim='variable').loc['s3'])[-1,:].data*Munit*Myr/(1024*1024)
        
    hh = hot+ion
    
    #print zprof_ds.taxis,cold,hh
    plt.plot(zprof_ds.taxis*Myr,cold,label='Cold/Warm',color=C2[0])
    plt.plot(zprof_ds.taxis*Myr,hh,label='Hot',color=C2[3])
    
    if j==0 or j==1 or j==2:
        hh=0.0062*0.85
    elif j==3:
        hh=0.152*0.85
    elif j==4:
        hh=0.23*0.85
    plt.text(250,hh,'%s' % labell[j],fontsize=19)
    
    icm_c = 'green'
    
    time=np.array(zprof_ds.taxis*Myr)
    
    if j == 0:
        plt.ylim(-0.00031,0.0062)
    elif j== 1: # ICM 0
        #plt.plot(time, icm, label='ICM outflow', color=icm_c,ls='--')
        icm_inflow = 0.0051002/4  # m_d_cuw + m_d_hi
        plt.axhline(icm_inflow,label='ICM inflow',ls=':',color='k')
        #plt.ylim(-0.000275, 0.0055)
        plt.ylim(-0.00031, 0.0062) # for 3 kpc
        #plt.ylim(-0.0325, 0.027) # for 2 kpc
    elif j== 2: # ICM 1
        #plt.plot(time, icm, label='ICM outflow', color=icm_c,ls='--')
        icm_inflow = 0.0051002 # m_d_cuw + m_d_hi
        plt.axhline(icm_inflow,label='ICM inflow',ls=':',color='k')
        #plt.ylim(-0.0175, 0.39)
        plt.ylim(-0.00031, 0.0062) # for 3 kpc
        #plt.ylim(-0.033, 0.131) # for 2 kpc
    elif j== 3: # ICM 2
        #plt.plot(time, icm, label='ICM outflow', color=icm_c,ls='--')
        icm_inflow = 0.0051002 * 2 # m_d_cuw + m_d_hi
        plt.axhline(icm_inflow,label='ICM inflow',ls=':',color='k')
        #plt.ylim(-0.0175, 0.39)
        plt.ylim(-0.00655, 0.151) # for 3 kpc
        #plt.ylim(-0.033, 0.131) # for 2 kpc
    elif j== 4: # ICM 3
        #plt.plot(time, icm, label='ICM outflow', color=icm_c,ls='--')
        icm_inflow = 0.0051002 * 4 # m_d_cuw + m_d_hi
        plt.axhline(icm_inflow,label='ICM inflow',ls=':',color='k')
        #plt.ylim(-0.0175, 0.39)
        plt.ylim(-0.00855, 0.23) # for 3 kpc
        #plt.ylim(-0.033, 0.131) # for 2 kpc
    
    
    if j==2:
        plt.legend(loc=0,fontsize=19)
        plt.ylabel(r'Mass flux [M$_{\odot}$ pc$^{-2}$ Myr$^{-1}$]', fontsize=24)
    if j==4:
        plt.xlabel(r'time [Myr]',fontsize=20)
        plt.xticks([250,300,350,400,450],fontsize=20)
    else :
        plt.xticks([250, 300, 350, 400, 450],[])
    plt.yticks(fontsize=19)
    plt.xlim(time[0],time[-1])
    plt.tick_params(direction='in')
    plt.locator_params(axis='y', nbins=6)
    '''
#plt.subplots_adjust(bottom=0.06,top=0.99,hspace = .01,left=0.17,right=0.99)

#plt.show()


Reading a history file:G:/yeongu//RPS_8pc_noICM_newacc/hst/RPS_8pc_noICM_newacc.hst.p
0.0352682242985 solMass / pc3
1.0 km2 / s2
21.3974898219 solMass / pc3 2.11098757754e-05 solMass / pc3
Reading a history file:G:/yeongu//RPS_8pc_ICM0_newacc/hst/RPS_8pc_ICM0_newacc.hst.p
0.0352682242985 solMass / pc3
1.0 km2 / s2
27.8280545276 solMass / pc3 0.000666867479846 solMass / pc3
Reading a history file:G:/yeongu//RPS_8pc_ICM1_newacc/hst/RPS_8pc_ICM1_newacc.hst.p
0.0352682242985 solMass / pc3
1.0 km2 / s2
21.5860845848 solMass / pc3 0.000294059841409 solMass / pc3
Reading a history file:G:/yeongu//RPS_8pc_ICM2_newacc/hst/RPS_8pc_ICM2_newacc.hst.p
0.0352682242985 solMass / pc3
1.0 km2 / s2
20.859104707 solMass / pc3 0.000757094085076 solMass / pc3
Reading a history file:G:/yeongu//RPS_8pc_ICM3_newacc/hst/RPS_8pc_ICM3_newacc.hst.p
0.0352682242985 solMass / pc3
1.0 km2 / s2
16.0519232952 solMass / pc3 0.00107607175355 solMass / pc3


IndexError: index 224 is out of bounds for axis 1 with size 224

In [84]:
r0 = 8000.
zz = 3500.
aa = r0**2*np.log(1+(zz/r0)**2)
print aa


11208597.2562
